# Creating datatypes

In [1]:
from recordclass import dataobject, clsconfig
from recordclass._linkedlist import linkedlist
import sys

## LinkedList

In [2]:
class LinkedItem(dataobject, fast_new=True):
    val: object
    next: 'LinkedItem'

@clsconfig(deep_dealloc=True)
class LinkedList(dataobject):
    start: LinkedItem = None
    end: LinkedItem = None
        
    def append(self, val):
        link = LinkedItem(val, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        self.end = link
        
#     def __del__(self):
#         curr = self.start
#         while curr is not None:
#             next = curr.next
#             curr.next = None
#             curr = next

    def __iter__(self):
        return IterLinkedList(self)

class IterLinkedList(dataobject, fast_new=True):
    node: LinkedItem
        
    def __init__(self, ll):
        self.node = ll.start
    
    def __next__(self):
        node =  self.node
        if node is None:
            raise StopIteration
        
        val = node.val
        self.node = node.next
        return val


In [3]:
def make_llist(N):
    ll = LinkedList()
    for i in range(N):
        ll.append(i)
    return ll

In [4]:
class LinkedItem2:
    __slots__ = 'val', 'next'

    def __init__(self, val, next):
        self.val = val
        self.next = next
    
class LinkedList2:
    __slots__ = 'start', 'end'
        
    def __init__(self, start=None, end=None):
        self.start = start
        self.end = end

    def append(self, val):
        link = LinkedItem2(val, None)
#         link.val = val
#         link.next = None
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        self.end = link

    def __iter__(self):
        return IterLinkedList2(self)

class IterLinkedList2:
    __slots__ = 'node',
    
    def __init__(self, ll):
        self.node = ll.start
    
    def __next__(self):
        node =  self.node
        if node is None:
            raise StopIteration
        
        val = node.val
        self.node = node.next
        return val

In [5]:
def make_llist2(N):
    ll = LinkedList2()
    for i in range(N):
        ll.append(i)
    return ll

In [19]:
N = 1000000
Mb = 1000000

In [20]:
%time ll1 = make_llist(N)

CPU times: user 261 ms, sys: 19.7 ms, total: 281 ms
Wall time: 279 ms


In [21]:
%time s1 = sum(ll1)
print(s1)

CPU times: user 138 ms, sys: 0 ns, total: 138 ms
Wall time: 136 ms
499999500000


In [22]:
print(sys.getsizeof(ll1.start), sys.getsizeof(ll1))
M1 = sys.getsizeof(ll1) + N * sys.getsizeof(ll1.start)
print('Memory footprint: %.2f Мб' % (M1/1000000))
# del ll1

32 32
Memory footprint: 32.00 Мб


In [23]:
%time ll2 = make_llist2(N)

CPU times: user 715 ms, sys: 11.7 ms, total: 727 ms
Wall time: 725 ms


In [24]:
%time s2 = sum(ll2)
print(s2)

CPU times: user 146 ms, sys: 78 µs, total: 146 ms
Wall time: 144 ms
499999500000


In [25]:
print(sys.getsizeof(ll2.start), sys.getsizeof(ll2))
M2 = sys.getsizeof(ll2) + N * sys.getsizeof(ll2.start)
print('Memory footprint: %.2f Мб' % (M2/1000000))
# del ll2

48 48
Memory footprint: 48.00 Мб


In [26]:
print(100*M1/M2)

66.66666666666667


In [27]:
def make_llist3(N):
    ll = linkedlist()
    for i in range(N):
        ll.append(i)
    return ll

In [28]:
%time ll3 = make_llist3(N)

CPU times: user 117 ms, sys: 27.9 ms, total: 145 ms
Wall time: 144 ms


In [29]:
%time s3 = sum(ll3)
print(s3)

CPU times: user 12.9 ms, sys: 0 ns, total: 12.9 ms
Wall time: 12.8 ms
499999500000


In [30]:
M3 = sys.getsizeof(ll3) + N * sys.getsizeof(ll3.start)
print('Memory footprint: %.2f Мб' % (M3/1000000))
# del ll3

Memory footprint: 32.00 Мб


In [31]:
# import random

# def test():
#     l = [1]
#     m = 10000000
#     k = 3
#     while l:
#         if m >= 0:
#             for v in range(k):
#                 l.append(v)
#             m -= 1
#         x = l.pop()